In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'wkmeans',
    'mapie',
    'causal',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='fractal', search_type='lgmm', search_subtype='',
        n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 5000
else:
    n_trials = len(study.trials) + 5000

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 133/5132 score=0.666931 avg=0.26s mem=644.42MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 134/5132 score=0.666931 avg=0.57s mem=663.39MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 135/5132 score=0.666931 avg=0.60s mem=593.16MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 136/5132 score=0.666931 avg=0.74s mem=701.33MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 137/5132 score=0.666931 avg=0.79s mem=658.64MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 138/5132 score=0.666931 avg=0.95s mem=708.67MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 139/5132 score=0.666931 avg=1.04s mem=748.14MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 140/5132 score=0.666931 avg=1.17s mem=712.20MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 141/5132 score=0.666931 avg=1.19s mem=714.26MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 142/5132 score=0.666931 avg=1.45s mem=738.74MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 143/5132 score=0.666931 avg=1.60s me